In [4]:
import os, numpy as np, uproot, awkward as ak
from scipy.stats import norm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import FloatSlider, VBox, HBox, Layout, HTML
from IPython.display import display

# your calculator API
from trackingerror import inputfromfile

# ---------------- config ----------------
ACTS_BASE = "/data/jlai/iris-hep/OutputPT_pixel_only"   # output_pt_<pT>/tracksummary_ckf.root
BASE_TXT  = "/data/jlai/iris-hep-log/TrackingResolution-3.0/TrackingResolution-3.0/myODD_best.txt"
TMP_TXT   = "./_tmp_ODD_slider.txt"

pT_values = np.arange(10, 100, 10)
var_labels = ['sigma(d)', 'sigma(z)', 'sigma(phi)', 'sigma(theta)', 'sigma(pt)/pt']

# slider ranges & step
WIDTH_RANGE = (-0.012, 0.040)     # Δwidth (x/X0)
RES_RANGE   = (-5e-6,  5e-6)      # Δres (m)
POS_RANGE   = (-5e-3,  5e-3)      # Δpos (m)
WIDTH_STEP, RES_STEP, POS_STEP = 5e-4, 1e-7, 1e-4

# colors (consistent across all subplots)
COLOR_ACTS = "#1f77b4"     # blue
COLOR_CALC = "#d62728"     # red

# -------------- helpers ---------------
def load_acts_results(base_dir, pts):
    y, yerr = {k: [] for k in var_labels}, {k: [] for k in var_labels}
    for pT in pts:
        f = uproot.open(os.path.join(base_dir, f"output_pt_{int(pT)}", "tracksummary_ckf.root"))
        arr = f["tracksummary"].arrays([
            "t_d0","eLOC0_fit","res_eLOC0_fit",
            "t_z0","eLOC1_fit","res_eLOC1_fit",
            "t_phi","ePHI_fit","res_ePHI_fit",
            "t_theta","eTHETA_fit","res_eTHETA_fit",
            "t_p","eQOP_fit","res_eQOP_fit","t_charge"
        ], library="ak")
        pT_truth = arr["t_p"] * np.sin(arr["t_theta"])
        pT_reco  = np.abs(1.0/arr["eQOP_fit"]) * np.sin(arr["t_theta"])
        samples = {
            'sigma(d)':     ak.flatten(arr['res_eLOC0_fit']) * 1e3,
            'sigma(z)':     ak.flatten(arr['res_eLOC1_fit']) * 1e3,
            'sigma(phi)':   ak.flatten(arr['res_ePHI_fit']),
            'sigma(theta)': ak.flatten(arr['res_eTHETA_fit']),
            'sigma(pt)/pt': ak.flatten((pT_reco - pT_truth) / pT_reco)
        }
        for k, v in samples.items():
            x = ak.to_numpy(v)
            x = x[np.isfinite(x)]
            if x.size < 3:
                sig, se = np.nan, np.nan
            else:
                _, sig = norm.fit(x)
                s = np.std(x, ddof=1)
                se = s / np.sqrt(2*max(x.size-1,1))
            y[k].append(sig); yerr[k].append(se)
    return y, yerr

def parse_best_txt(path):
    widths, resxy, resz, pos = [], None, None, []
    beam = None
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"): continue
            vals = line.split()
            if len(vals) != 4: continue
            w, rxy, rz, p = map(float, vals)
            if beam is None:
                beam = (w, rxy, rz, p)  # beam pipe line
            else:
                widths.append(w); pos.append(p)
                resxy = rxy if resxy is None else resxy
                resz  = rz  if resz  is None else resz
    if len(widths) != 4: raise RuntimeError("Expected 4 pixel layers in myODD_best.txt")
    return beam, np.array(widths,float), float(resxy), float(resz), np.array(pos,float)

def write_config_from_base(base_path, out_path, d_width, d_res, d_pos):
    beam, widths, rxy, rz, pos = parse_best_txt(base_path)
    widths2 = np.clip(widths + d_width, 0.0, 0.2)
    rxy2    = max(rxy + d_res, 1e-9)
    rz2     = max(rz  + d_res, 1e-9)
    pos2    = pos + d_pos
    with open(out_path, "w") as f:
        f.write("# width(x/X0)  resolutionxy(m)  resolutionz(m)  position (m)\n")
        f.write("# beam pipe\n")
        f.write(f"{beam[0]} {beam[1]} {beam[2]} {beam[3]}\n")
        f.write("# pixel\n")
        for i in range(4):
            f.write(f"{widths2[i]:.6f} {rxy2:.9e} {rz2:.9e} {pos2[i]:.6f}\n")

def calc_for_config(txt_path, pts):
    out = {k: [] for k in var_labels}
    for pT in pts:
        det = inputfromfile(txt_path, 0)
        res = det.errorcalculation(float(pT), 2.0, 0.0, 0.106)
        for k in var_labels:
            out[k].append(res[k])
    return out

# -------------- load ACTS once --------------
print("Loading ACTS curves…")
acts_y, acts_err = load_acts_results(ACTS_BASE, pT_values)

# -------------- figure (single legend) --------------
rmap = {0:(1,1), 1:(1,2), 2:(1,3), 3:(2,1), 4:(2,2)}  # (2,3) left empty
fig = make_subplots(rows=2, cols=3,
                    specs=[[{}, {}, {}], [{}, {}, None]],
                    subplot_titles=var_labels)

# show only two legend items total
legend_shown = {"ACTS": False, "Calculator": False}

trace_idx_calc = {}
for i, label in enumerate(var_labels):
    r, c = rmap[i]

    # ACTS (fixed)
    show_leg = not legend_shown["ACTS"]; legend_shown["ACTS"] = True
    fig.add_trace(
        go.Scatter(
            x=pT_values, y=acts_y[label],
            mode="lines+markers", name="ACTS", legendgroup="ACTS", showlegend=show_leg,
            error_y=dict(type="data", array=acts_err[label], visible=True),
            marker_symbol="x", marker_color=COLOR_ACTS, line_color=COLOR_ACTS
        ),
        row=r, col=c
    )

    # Calculator (updated by sliders)
    show_leg = not legend_shown["Calculator"]; legend_shown["Calculator"] = True
    tr = go.Scatter(
        x=pT_values, y=[np.nan]*len(pT_values),
        mode="lines+markers", name="Calculator", legendgroup="Calculator", showlegend=show_leg,
        marker_color=COLOR_CALC, line_color=COLOR_CALC
    )
    fig.add_trace(tr, row=r, col=c)
    trace_idx_calc[label] = len(fig.data) - 1

# log y on all used subplots
for ax in ["yaxis","yaxis2","yaxis3","yaxis4","yaxis5"]:
    fig.layout[ax].type = "log"

fig.update_layout(
    height=700, width=1100,
    title_text="Tracking Resolution: ACTS (fixed) vs Calculator (sliders)",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# -------------- sliders with precise readout --------------
s_width = FloatSlider(
    description="Δwidth (x/X0)", value=0.0, min=WIDTH_RANGE[0], max=WIDTH_RANGE[1],
    step=WIDTH_STEP, readout=True, readout_format=".6f",  # <= precise fixed-point
    layout=Layout(width="360px")
)
s_res = FloatSlider(
    description="Δres (m)", value=0.0, min=RES_RANGE[0], max=RES_RANGE[1],
    step=RES_STEP, readout=True, readout_format=".2e",   # <= scientific for tiny nums
    layout=Layout(width="360px")
)
s_pos = FloatSlider(
    description="Δpos (m)", value=0.0, min=POS_RANGE[0], max=POS_RANGE[1],
    step=POS_STEP, readout=True, readout_format=".6f",   # positions ~1e-3 → show 6 dp
    layout=Layout(width="360px")
)

status = HTML(value="<span style='font-size:17px'>Move sliders to update calculator…")

def recompute_and_update(*_):
    status.value = "<b>Updating…</b>"
    try:
        write_config_from_base(
            BASE_TXT, TMP_TXT,
            d_width=s_width.value, d_res=s_res.value, d_pos=s_pos.value
        )
        y_calc = calc_for_config(TMP_TXT, pT_values)
        for label in var_labels:
            idx = trace_idx_calc[label]
            fig.data[idx].y = y_calc[label]
        status.value = ("<span style='font-size:17px'>"
            f"Δwidth={s_width.value:+.6f}  |  "
            f"Δres={s_res.value:+.2e} m  |  "
            f"Δpos={s_pos.value:+.6f} m"
        )
    except Exception as e:
        status.value = f"<span style='color:red'>Update failed: {e}</span>"

# initial compute + wire up
recompute_and_update()
s_width.observe(recompute_and_update, names="value")
s_res.observe(recompute_and_update, names="value")
s_pos.observe(recompute_and_update, names="value")

fig = go.FigureWidget(fig)   # wrap the figure as a widget
display(VBox([HBox([s_width, s_res, s_pos]), status, fig]))

Loading ACTS curves…


In [5]:
import plotly.io as pio

# `fig` is your current Plotly figure (already updated by sliders in the notebook)
html_snippet = pio.to_html(
    fig,
    include_plotlyjs="cdn",   # no big inline JS bundle
    full_html=False,          # give me just the <div> + small <script>
    div_id="tracking_res_plot"  # optional: stable id if you have multiple figures
)

with open("tracking_res_plot_snippet.html", "w") as f:
    f.write(html_snippet)


In [6]:
import os, json, numpy as np, uproot, awkward as ak
from scipy.stats import norm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# ---- your calculator API ----
from trackingerror import inputfromfile

# ---- CONFIG (edit these) ----
ACTS_BASE = "/data/jlai/iris-hep/OutputPT_pixel_only"   # contains output_pt_<pT>/tracksummary_ckf.root
BASE_TXT  = "/data/jlai/iris-hep-log/TrackingResolution-3.0/TrackingResolution-3.0/myODD_best.txt"
TMP_TXT   = "./_tmp_ODD_slider_export.txt"
OUT_HTML  = "./tracking_res_with_sliders.html"

pT_values = np.arange(10, 100, 10)
var_labels = ['sigma(d)', 'sigma(z)', 'sigma(phi)', 'sigma(theta)', 'sigma(pt)/pt']

# Discrete slider positions (edit freely)
W_LIST = [-0.010, -0.005, 0.0, 0.005, 0.010]         # Δwidth (x/X0)
R_LIST = [-2e-6, -1e-6, 0.0, 1e-6, 2e-6]             # Δres (m)
P_LIST = [-1e-3, -5e-4, 0.0, 5e-4, 1e-3]             # Δpos (m)

# Colors
COLOR_ACTS = "#1f77b4"
COLOR_CALC = "#d62728"

# ---------- helpers ----------
def load_acts_results(base_dir, pts):
    y, yerr = {k: [] for k in var_labels}, {k: [] for k in var_labels}
    for pT in pts:
        f = uproot.open(os.path.join(base_dir, f"output_pt_{int(pT)}", "tracksummary_ckf.root"))
        arr = f["tracksummary"].arrays([
            "t_d0","eLOC0_fit","res_eLOC0_fit",
            "t_z0","eLOC1_fit","res_eLOC1_fit",
            "t_phi","ePHI_fit","res_ePHI_fit",
            "t_theta","eTHETA_fit","res_eTHETA_fit",
            "t_p","eQOP_fit","res_eQOP_fit","t_charge"
        ], library="ak")
        pT_truth = arr["t_p"] * np.sin(arr["t_theta"])
        pT_reco  = np.abs(1.0/arr["eQOP_fit"]) * np.sin(arr["t_theta"])
        samples = {
            'sigma(d)':     ak.flatten(arr['res_eLOC0_fit']) * 1e3,
            'sigma(z)':     ak.flatten(arr['res_eLOC1_fit']) * 1e3,
            'sigma(phi)':   ak.flatten(arr['res_ePHI_fit']),
            'sigma(theta)': ak.flatten(arr['res_eTHETA_fit']),
            'sigma(pt)/pt': ak.flatten((pT_reco - pT_truth) / pT_reco)
        }
        for k, v in samples.items():
            x = ak.to_numpy(v)
            x = x[np.isfinite(x)]
            if x.size < 3:
                sig, se = np.nan, np.nan
            else:
                _, sig = norm.fit(x)
                s = np.std(x, ddof=1)
                se = s / np.sqrt(2*max(x.size-1,1))
            y[k].append(float(sig)); yerr[k].append(float(se))
    return y, yerr

def parse_best_txt(path):
    widths, resxy, resz, pos = [], None, None, []
    beam = None
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"): continue
            vals = line.split()
            if len(vals) != 4: continue
            w, rxy, rz, p = map(float, vals)
            if beam is None:
                beam = (w, rxy, rz, p)  # beam pipe
            else:
                widths.append(w); pos.append(p)
                resxy = rxy if resxy is None else resxy
                resz  = rz  if resz  is None else resz
    if len(widths) != 4:
        raise RuntimeError("Expected 4 pixel layers in myODD_best.txt")
    return beam, np.array(widths,float), float(resxy), float(resz), np.array(pos,float)

def write_config_from_base(base_path, out_path, d_width, d_res, d_pos):
    beam, widths, rxy, rz, pos = parse_best_txt(base_path)
    widths2 = np.clip(widths + d_width, 0.0, 0.2)
    rxy2    = max(rxy + d_res, 1e-9)
    rz2     = max(rz  + d_res, 1e-9)
    pos2    = pos + d_pos
    with open(out_path, "w") as f:
        f.write("# width(x/X0)  resolutionxy(m)  resolutionz(m)  position (m)\n")
        f.write("# beam pipe\n")
        f.write(f"{beam[0]} {beam[1]} {beam[2]} {beam[3]}\n")
        f.write("# pixel\n")
        for i in range(4):
            f.write(f"{widths2[i]:.6f} {rxy2:.9e} {rz2:.9e} {pos2[i]:.6f}\n")

def calc_for_config(txt_path, pts):
    out = {k: [] for k in var_labels}
    for pT in pts:
        det = inputfromfile(txt_path, 0)
        res = det.errorcalculation(float(pT), 2.0, 0.0, 0.106)
        for k in var_labels:
            out[k].append(float(res[k]))
    return out

# ---------- precompute ----------
print("Loading ACTS…")
acts_y, acts_err = load_acts_results(ACTS_BASE, pT_values)

print("Precomputing calculator grid…")
calc_grid = {}  # key "i|j|k" -> dict(var -> list of y)
for i, dw in enumerate(W_LIST):
    for j, dr in enumerate(R_LIST):
        for k, dp in enumerate(P_LIST):
            write_config_from_base(BASE_TXT, TMP_TXT, dw, dr, dp)
            y_calc = calc_for_config(TMP_TXT, pT_values)
            calc_grid[f"{i}|{j}|{k}"] = y_calc

# ---------- build Plotly fig (initial at center indices) ----------
iw0, ir0, ip0 = len(W_LIST)//2, len(R_LIST)//2, len(P_LIST)//2
init_key = f"{iw0}|{ir0}|{ip0}"
init_calc = calc_grid[init_key]

rmap = {0:(1,1), 1:(1,2), 2:(1,3), 3:(2,1), 4:(2,2)}  # leave (2,3) empty
fig = make_subplots(rows=2, cols=3,
                    specs=[[{}, {}, {}], [{}, {}, None]],
                    subplot_titles=var_labels)

legend_shown = {"ACTS": False, "Calculator": False}
calc_trace_indices = []

for idx, label in enumerate(var_labels):
    r, c = rmap[idx]
    show_leg = not legend_shown["ACTS"]; legend_shown["ACTS"] = True
    fig.add_trace(
        go.Scatter(
            x=pT_values, y=acts_y[label],
            mode="lines+markers", name="ACTS", legendgroup="ACTS", showlegend=show_leg,
            error_y=dict(type="data", array=acts_err[label], visible=True),
            marker_symbol="x", marker_color=COLOR_ACTS, line_color=COLOR_ACTS
        ), row=r, col=c
    )
    show_leg = not legend_shown["Calculator"]; legend_shown["Calculator"] = True
    tr = go.Scatter(
        x=pT_values, y=init_calc[label],
        mode="lines+markers", name="Calculator", legendgroup="Calculator", showlegend=show_leg,
        marker_color=COLOR_CALC, line_color=COLOR_CALC
    )
    fig.add_trace(tr, row=r, col=c)
    calc_trace_indices.append(len(fig.data) - 1)

for ax in ["yaxis","yaxis2","yaxis3","yaxis4","yaxis5"]:
    fig.layout[ax].type = "log"

fig.update_layout(
    height=720, width=1100,
    title_text="Tracking Resolution: ACTS (fixed) vs Calculator (HTML sliders)",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# ---------- export lightweight HTML with our own sliders ----------
snippet = pio.to_html(
    fig, include_plotlyjs=False, full_html=False, div_id="tracking_res_plot"
)

html = f"""<!doctype html>
<meta charset="utf-8">
<title>Tracking Resolution (ACTS vs Calculator)</title>
<link rel="preconnect" href="https://cdn.plot.ly">
<script src="https://cdn.plot.ly/plotly-2.35.2.min.js"></script>

<style>
  body {{ font: 14px/1.35 system-ui, -apple-system, Segoe UI, Roboto, sans-serif; margin: 16px; }}
  .ctrls label {{ display:inline-block; min-width: 210px; margin-right: 8px; }}
  .ctrls input[type=range] {{ width: 280px; vertical-align: middle; }}
  .ctrls .val {{ display:inline-block; width: 110px; font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; text-align:right; }}
  .note {{ color:#555; margin:6px 0 14px; }}
</style>

<div class="ctrls">
  <div>
    <label>Δwidth (x/X0):
      <input id="w" type="range" min="0" max="{len(W_LIST)-1}" step="1" value="{iw0}">
    </label>
    <span class="val" id="wval"></span>
  </div>
  <div>
    <label>Δres (m):
      <input id="r" type="range" min="0" max="{len(R_LIST)-1}" step="1" value="{ir0}">
    </label>
    <span class="val" id="rval"></span>
  </div>
  <div>
    <label>Δpos (m):
      <input id="p" type="range" min="0" max="{len(P_LIST)-1}" step="1" value="{ip0}">
    </label>
    <span class="val" id="pval"></span>
  </div>
  <div class="note">Move the sliders: ACTS stays fixed; Calculator updates (precomputed grid).</div>
</div>

{snippet}

<script>
const LABELS = {json.dumps(var_labels)};
const CALC_TRACE_IDX = {json.dumps(calc_trace_indices)};  // which traces we update

// slider value lists (index -> delta displayed)
const W_LIST = {json.dumps(W_LIST)};
const R_LIST = {json.dumps(R_LIST)};
const P_LIST = {json.dumps(P_LIST)};

// precomputed calculator values for all combinations
//   key "i|j|k" -> {{ "<var_label>": [.. per pT ..], ... }}
const CALC_GRID = {json.dumps(calc_grid)};

function update() {{
  const iw = +document.getElementById('w').value;
  const ir = +document.getElementById('r').value;
  const ip = +document.getElementById('p').value;

  // readouts
  document.getElementById('wval').textContent = W_LIST[iw].toFixed(6);
  document.getElementById('rval').textContent = Number(R_LIST[ir]).toExponential(2);
  document.getElementById('pval').textContent = P_LIST[ip].toFixed(6);

  const key = iw + '|' + ir + '|' + ip;
  const c   = CALC_GRID[key];
  const gd  = document.getElementById('tracking_res_plot');

  // Update each calculator trace (one per panel) with the new y
  CALC_TRACE_IDX.forEach((tIdx, n) => {{
    const y = c[LABELS[n]];
    Plotly.restyle(gd, {{y: [y]}}, [tIdx]);
  }});
}}

// wire up
['w','r','p'].forEach(id => document.getElementById(id).addEventListener('input', update));
update();  // initial
</script>
"""

with open(OUT_HTML, "w") as f:
    f.write(html)

print(f"Saved: {OUT_HTML}")


Loading ACTS…
Precomputing calculator grid…
Saved: ./tracking_res_with_sliders.html
